<a href="https://colab.research.google.com/github/Asif1310/3009/blob/main/Cricketanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.mkdir('/content/json_data') # to create a directory to store all json files

In [2]:
# downloading zip file from the URL and extracting it to the new directory created in previous step
import zipfile 
!wget -O '/content/json_data/main.zip' 'https://cricsheet.org/downloads/t20s_json.zip'
zip1 = zipfile.ZipFile('/content/json_data/main.zip','r')
zip1.extractall('/content/json_data')
zip1.close()

--2022-07-02 14:33:21--  https://cricsheet.org/downloads/t20s_json.zip
Resolving cricsheet.org (cricsheet.org)... 64.90.49.16
Connecting to cricsheet.org (cricsheet.org)|64.90.49.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6504477 (6.2M) [application/zip]
Saving to: ‘/content/json_data/main.zip’

/content/json_data/ 100%[===================>]   6.20M  5.86MB/s    in 1.1s    

2022-07-02 14:33:23 (5.86 MB/s) - ‘/content/json_data/main.zip’ saved [6504477/6504477]



In [3]:
# removing the main ZIP file and a README.txt from the directory
!rm /content/json_data/README.txt
!rm /content/json_data/main.zip

In [4]:
import json
import pandas as pd
import numpy as np

#initializing an empty dataframe
df = pd.DataFrame(columns = ['city','date','venue', 'gender','team_1','team_2','toss_winner','toss_decision','winner','target','margin','victory_margin','MOM','referee','tv_umpire','field_umpire_1','field_umpire_2'])


In [5]:
for j,i in enumerate(os.listdir('/content/json_data')): #looping through all the JSON files in the folder
  g = '/content/json_data/' + i
  data = json.load(open(g))
  try:
    df.loc[j,'city'] = data['info']['city']
  except KeyError:
    df.loc[j,'city'] = np.NaN
  df.loc[j,'date'] = data['info']['dates'][0]
  try:
    df.loc[j,'winner'] = data['info']['outcome']['winner'] 
  except KeyError: 
    try:
      df.loc[j,'winner'] = data['info']['outcome']['eliminator'] #tie-eliminator
    except KeyError:
      df.loc[j,'winner'] = data['info']['outcome']['result'] # no result
  try:
    df.loc[j,'margin'] = str(data['info']['outcome']['by'])
  except KeyError:
    try:
      if data['info']['outcome']['result'] == 'tie':
        df.loc[j,'margin'] = 'Tie-eliminator'
      elif data['info']['outcome']['result'] == 'no result':
        df.loc[j,'margin'] = np.NaN
    except KeyError:
      df.loc[j,'margin'] = data['info']['outcome']['method']
  try:
    df.loc[j,'MOM'] = data['info']['player_of_match'][0]
  except KeyError:
    df.loc[j,'MOM'] = np.NaN
  try:
    df.loc[j,'referee'] = data['info']['officials']['match_referees'][0]
  except KeyError:
    df.loc[j,'referee'] = np.NaN
  try:
    df.loc[j,'tv_umpire'] = data['info']['officials']['tv_umpires'][0]
  except KeyError:
    df.loc[j,'tv_umpire'] = np.NaN
  try:
    df.loc[j,'field_umpire_1'] = data['info']['officials']['umpires'][0]
  except KeyError:
    df.loc[j,'field_umpire_1'] = np.NaN
  try:
    df.loc[j,'field_umpire_2'] = data['info']['officials']['umpires'][1]
  except KeyError:
    df.loc[j,'field_umpire_2'] = np.NaN
  list1 = []
  for key, value in data['info']['players'].items():
    list1.append(key)
  df.loc[j, 'team_1'] = list1[0]
  df.loc[j, 'team_2'] = list1[1]
  df.loc[j, 'gender'] = data['info']['gender']
  df.loc[j, 'toss_winner'] = data['info']['toss']['winner']
  df.loc[j, 'toss_decision'] =  data['info']['toss']['decision']
  df.loc[j, 'venue'] = data['info']['venue']
  try:
    df.loc[j, 'target'] = data['innings'][1]['target']['runs']
  except KeyError:
    df.loc[j, 'target'] = np.NaN
  except IndexError:
    df.loc[j, 'target'] = np.NaN
  data.clear()



In [6]:
df['margin'] = df['margin'].str.strip('{}') # transforming dictionaries in margin column to a proper string format
df['margin'] = df['margin'].str.replace(':','') # transforming dictionaries in margin column to a proper string format
df['margin'] = df['margin'].str.replace('\'','') # transforming dictionaries in margin column to a proper string format
df_new = df['margin'].str.split(' ', expand = True) 
df_new[0] = df_new[0].str.replace('\'','')
df['victory_margin'] = df_new[1] + ' ' + df_new[0] # assigning transformed strings to the victory_margin column
df.drop(['margin'],axis = 1, inplace = True) #dropping the redundant margin column


In [7]:
df.head()

,city,date,venue,gender,team_1,team_2,toss_winner,toss_decision,winner,target,victory_margin,MOM,referee,tv_umpire,field_umpire_1,field_umpire_2
0,Harare,2019-05-05,Old Hararians,female,Uganda,Sierra Leone,Uganda,bat,Uganda,144,90 runs,D Busingye,O Chirombe,NaN,IO Oyieko,A Louw
1,Bangi,2022-06-22,"UKM-YSD Cricket Oval, Bangi",female,Hong Kong,Kuwait,Kuwait,field,Hong Kong,91,30 runs,M Bibi,Ramesh Menon,NaN,L Poobalan,V Kalidas
2,Windhoek,2019-08-23,United Cricket Club Ground,male,Botswana,Namibia,Botswana,bat,Namibia,86,8 wickets,MG Erasmus,NaN,NaN,JJ Luck,Marthinus Louw
3,Prague,2021-05-23,"Scott Page Field, Vinor",male,Czech Republic,Austria,Czech Republic,bat,Austria,159,4 wickets,NaN,NaN,NaN,J Foster,S Tripp
4,Harare,2019-05-05,Old Hararians,female,Zimbabwe,Mozambique,Zimbabwe,bat,Zimbabwe,199,163 runs,SM Mayers,O Chirombe,NaN,A Louw,R D'Mello


In [8]:
df.dtypes

city              object
date              object
venue             object
gender            object
team_1            object
team_2            object
toss_winner       object
toss_decision     object
winner            object
target            object
victory_margin    object
MOM               object
referee           object
tv_umpire         object
field_umpire_1    object
field_umpire_2    object
dtype: object

In [9]:
df_players = pd.DataFrame(columns = ['venue','date','T1','T2']) # A new dataframe to store playing XI details

In [10]:
for j,i in enumerate(os.listdir('/content/json_data')): #looping through all the JSON files in the folder
  player_list = []
  g = '/content/json_data/' + i
  data = json.load(open(g))
  df_players.loc[j, 'venue'] = data['info']['venue']
  df_players.loc[j,'date'] = data['info']['dates'][0]
  for key, value in data['info']['players'].items():
    list1.append(key)
    for player in value:
      player_list.append(player)
  df_players.loc[j,'T1P1'] = player_list[0]
  df_players.loc[j,'T1P2'] = player_list[1]
  df_players.loc[j,'T1P3'] = player_list[2]
  df_players.loc[j,'T1P4'] = player_list[3]
  df_players.loc[j,'T1P5'] = player_list[4]
  df_players.loc[j,'T1P6'] = player_list[5]
  df_players.loc[j,'T1P7'] = player_list[6]
  df_players.loc[j,'T1P8'] = player_list[7]
  df_players.loc[j,'T1P9'] = player_list[8]
  df_players.loc[j,'T1P10'] = player_list[9]
  df_players.loc[j,'T1P11'] = player_list[10]
  df_players.loc[j,'T1P12'] = player_list[11]
  df_players.loc[j,'T1P13'] = player_list[12]
  df_players.loc[j,'T1P14'] = player_list[13]
  df_players.loc[j,'T1P15'] = player_list[14]
  df_players.loc[j,'T1P16'] = player_list[15]
  df_players.loc[j,'T1P17'] = player_list[16]
  df_players.loc[j,'T1P18'] = player_list[17]
  df_players.loc[j,'T1P19'] = player_list[18]
  df_players.loc[j,'T1P20'] = player_list[19]
  df_players.loc[j,'T1P21'] = player_list[20]
  df_players.loc[j,'T1P22'] = player_list[21]  
  df_players.loc[j, 'T1'] = list1[0]
  df_players.loc[j, 'T2'] = list1[1]

In [11]:
df_players.head()

,venue,date,T1,T2,T1P1,T1P2,T1P3,T1P4,T1P5,T1P6,...,T1P13,T1P14,T1P15,T1P16,T1P17,T1P18,T1P19,T1P20,T1P21,T1P22
0,Old Hararians,2019-05-05,South Africa,England,R Ntono,D Busingye,NK Bagenda,F Najjumba,K Awino,I Nakisuuyi,...,M Sankoh,Aminata Kamara,L Bull,Adama Kamara,AM Kamara,Z Kamara,I Koroma,J Kowa,M Sheriff,F Bangur
1,"UKM-YSD Cricket Oval, Bangi",2022-06-22,South Africa,England,Bella Poon,M Hill,NT Miles,KY Chan,Shanzeen Shahzad,Y Daswani,...,Priyada Murali,Zeefa Jilani,Amna Tariq,SL Gomez,Khadija Khalil,Mariamma Hyder,B Shanti,Maria Jasvi,Maryyam Ashraf,A Bose
2,United Cricket Club Ground,2019-08-23,South Africa,England,T Perera,K Mothlhanka,Ameer Saiyed,R Nehonde,V Mbazo,H Pragji,...,JP Kotze,MG Erasmus,L Louwrens,JN Frylinck,HN Ya France,ZE Green,BM Scholtz,Z Groenewald,CG Williams,B Shikongo
3,"Scott Page Field, Vinor",2021-05-23,South Africa,England,S Davizi,EB Knowles,A Ashokan,S Wickramasekara,K Mendon,S Grover,...,M Simpson-Parker,Razmal Shigiwal,I Asif,N Wijesekera,Mirza Ahsan,Aqib Iqbal,LP Kasturiarachchige,Sahel Zadran,Rayhaan Ahmed,Umair Tariq
4,Old Hararians,2019-05-05,South Africa,England,M Mupachikwa,SM Mayers,CS Mugeri,M Musonda,J Nkomo,AC Mushangwe,...,M Matine,R Haiong,C Murrombe,P Cuinica,F Guirrugo,J Sainda,C Magaia,I Chuma,O Matsolo,E Moiane


In [12]:
list2 = []
for i in os.listdir('/content/json_data'): #looping through all the JSON files in the folder
  g = '/content/json_data/' + i
  data = json.load(open(g))
  date = data['info']['dates'][0]
  venue = data['info']['venue']
  for inning in data['innings']:
    team = inning['team']
    for over in (inning['overs']):
      over_num = over['over']
      ball_counter = 0
      for ball in over['deliveries']:
        list1 = []
        ball_counter += 1
        batter = ball['batter']
        bowler = ball['bowler']
        non_striker = ball['non_striker']
        batter_runs = ball['runs']['batter']
        extra_runs = ball['runs']['extras']
        total_runs = ball['runs']['total']
        team = team
        over_num = over_num
        try:
          for key,value in ball['extras'].items():
            extra_type = key
        except KeyError:
          extra_type = 'NA'
        try:
          wicket_type = ball['wickets'][0]['kind']
        except KeyError:
          wicket_type = 'NA'
        try:
          player_out = ball['wickets'][0]['player_out']
        except KeyError:
          player_out = 'NA'
        try:
          fielder = ball['wickets'][0]['fielders'][0]['name']
        except KeyError:
          fielder = 'NA'
        list1 = [date, venue, team, over_num, ball_counter, batter, bowler, non_striker,batter_runs,extra_runs,extra_type,total_runs,wicket_type,player_out,fielder]
        list2.append(list1)



In [13]:
df_main = pd.DataFrame(list2, columns = ['date', 'venue','team', 'over', 'ball_num','batter','bowler','non_striker','batter_runs','extra_runs','extra_type','total_runs', 'wicket_type', 'player_out', 'fielder'])

In [14]:
df_main[df_main['wicket_type']!= 'NA']

,date,venue,team,over,ball_num,batter,bowler,non_striker,batter_runs,extra_runs,extra_type,total_runs,wicket_type,player_out,fielder
7,2019-05-05,Old Hararians,Uganda,1,1,R Ntono,J Kowa,D Busingye,0,0,NA,0,lbw,R Ntono,NA
84,2019-05-05,Old Hararians,Uganda,12,6,NK Bagenda,AM Kamara,D Busingye,0,0,NA,0,bowled,NK Bagenda,NA
162,2019-05-05,Old Hararians,Sierra Leone,4,5,M King,C Aweko,M Sankoh,0,0,NA,0,bowled,M King,NA
163,2019-05-05,Old Hararians,Sierra Leone,4,6,Aminata Kamara,C Aweko,M Sankoh,0,0,NA,0,run out,Aminata Kamara,NA
168,2019-05-05,Old Hararians,Sierra Leone,5,4,M Sankoh,R Musamali,L Bull,0,0,NA,0,bowled,M Sankoh,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466009,2021-11-06,Sharjah Cricket Stadium,England,16,2,DJ Malan,D Pretorius,LS Livingstone,0,0,NA,0,caught,DJ Malan,K Rabada
466020,2021-11-06,Sharjah Cricket Stadium,England,18,1,LS Livingstone,D Pretorius,EJG Morgan,0,0,NA,0,caught,LS Livingstone,DA Miller
466027,2021-11-06,Sharjah Cricket Stadium,England,19,1,CR Woakes,K Rabada,EJG Morgan,0,0,NA,0,caught,CR Woakes,A Nortje
466028,2021-11-06,Sharjah Cricket Stadium,England,19,2,EJG Morgan,K Rabada,CJ Jordan,0,0,NA,0,caught,EJG Morgan,KA Maharaj


In [15]:
df_main.dtypes

date           object
venue          object
team           object
over            int64
ball_num        int64
batter         object
bowler         object
non_striker    object
batter_runs     int64
extra_runs      int64
extra_type     object
total_runs      int64
wicket_type    object
player_out     object
fielder        object
dtype: object